# Problem (from notebooks/api_search_recs_pkl_parquet_sqlite)
- search_recs write/read with either .sqlite/.parquet is fast
- But I forgot to account for the _~20s_ it takes to convert back _to_ search_recs from the serialized df

# Solution: save/load separate files
- feats.npy for heavy np.array cols + non_feats.parquet for normal df cols
- A little more complexity to manage, but an ok tradeoff
- Conclusion: ~1s read, ~3s write

In [ ]:
from notebooks import *
sg.init(None, skip=[sg_load.load_search_recs])
get_search_recs.cache_clear()

INFO     [13:34:24.324] [42810]  52 logging_/init_logging: {logging_yaml: None}

INFO     [13:34:24.626] [42810]  32 api.server_globals/init

DEBUG    [13:34:24.633] [42810]  75 api.server_globals/load: load_search...

INFO     [13:34:24.909] [42810]  79 api.server_globals/load: load_search (took 0.270s)

DEBUG    [13:34:24.916] [42810]  75 api.server_globals/load: load_xc_meta...

...

DEBUG    [13:34:25.331] [42810]  75 api.server_globals/load: load_feat_info...

INFO     [13:34:25.338] [42810] 110 api.server_globals/load_feat_info

INFO     [13:34:25.343] [42810]  79 api.server_globals/load: load_feat_info (took 0.005s)

WARNING  [13:34:25.349] [42810]  73 api.server_globals/load: load_search_recs [skipped]

INFO     [13:34:25.355] [42810]  34 api.server_globals/init: done

In [ ]:
kwargss = [
    dict(cache_type='hybrid', refresh=True),  # Simulate cache miss
    dict(cache_type='hybrid'),                # Simulate cache hit
]
search_recss = []
for kwargs in kwargss:
    print()
    print(f'search_recs(**{kwargs})')
    get_search_recs.cache_clear()
    search_recs = get_search_recs(**kwargs)
    search_recss.append(search_recs)
    display(search_recs
        [['xc_id', 'feat', 'f_preds', 'background', 'background_species']]
        [2:3].reset_index(drop=True).T
        .pipe(df_assign_first,
            size=lambda df: df.T.memory_usage(deep=True),
            type=lambda df: df[0].map(lambda x: type(x).__name__),
        )
    )

search_recs(**{'cache_type': 'hybrid', 'refresh': True})

INFO     [13:34:25.538] [42810] 314 api.recs/get_search_recs

INFO     [13:34:25.551] [42810]  61 payloads/df_cache_hybrid: Miss: payloads/search_recs-version[0],audio_s[10],countries_k[na],com_names_k[ca],num_recs[None]-b89a9fc

df_cache_hybrid:compute[search_recs]:   0%|                                                      | 0/1 [00:00<?, ?it/s]

INFO     [13:34:25.560] [42810] 371 api.recs/_compute_search_recs: {len(sg.xc_meta): 35233, countries_k: na, com_names_k: ca, num_recs: null}

...

INFO     [13:34:41.543] [42810] 142 payloads/df_cache_hybrid: Hit: Read feat-feat.npy: float32 (211.4 MB)

INFO     [13:34:41.578] [42810] 142 payloads/df_cache_hybrid: Hit: Read feat-f_preds.npy: float32 (46.6 MB)

INFO     [13:34:41.586] [42810] 146 payloads/df_cache_hybrid: Hit: Join non_feats + feats

INFO     [13:34:41.755] [42810] 160 payloads/df_cache_hybrid: Hit: Done (took: 2.056s)

,size,type,0
xc_id,36,int,115925
feat,104,ndarray,[ 4.455e-02 -7.034e-02 2.956e-01 -4.216e-02 -3.264e-02 -6.938e-02 9.214e-02 3.209e-03 2.126e-01 -8.443e-02 -1.446e-01 -9.171e-02 -1.280e-01 1.579e-02 -7.380e-02 -7.01...
f_preds,104,ndarray,[2.792e-04 2.333e-03 6.595e-06 7.704e-04 4.692e-04 5.952e-03 1.236e-02 4.282e-03 2.968e-05 2.268e-03 1.762e-04 2.358e-04 3.394e-03 2.391e-03 3.478e-03 2.690e-03 3.178e-04 ...
background,80,list,['Canada Goose (Branta canadensis)']
background_species,80,list,['CANG']


In [ ]:
# TODO Turn this into unit tests
#   - Should roundtrip, should preserve category dtypes, should fail if index present
[a, b] = search_recss
pd.testing.assert_frame_equal(a, b, check_index_type=True, check_column_type=True)